# Homework 4 Part 2 - Loss Visualization

## Course Name: Large Language Models
#### Lecturers: Dr. Soleimani, Dr. Rohban, Dr. Asgari

---

#### Notebooks Supervised By: MohammadAli SadraeiJavaheri
#### Notebooks Prepared By: Ali Razghandi, Mahdi Zakizadeh, Faridoun Mehri

**Contact**: Ask your questions in Quera

---

### Instructions:
- Complete all exercises presented in this notebook.
- Ensure you run each cell after you've entered your solution.
- After completing the exercises, save the notebook and <font color='red'>follow the submission guidelines provided in the PDF.</font>


---

**Note**: Fill in the `#TODO` sections with the correct code to complete the exercise.

In this exercise, you will explore the loss landscape of a pre-trained language model (GPT-2) using PyTorch. The goal is to gain an intuitive understanding of how the model\'s parameters affect its performance and how sensitive the model is to changes in different directions of the parameter space. The exercise will guide you through the following steps:

1.  **Setting Up the Environment**: You\'ll start by importing necessary libraries such as `torch`, `numpy`, `plotly` for visualization, `tqdm` for progress bars, and `imageio` for image processing. You\'ll also need to handle file operations with `os`.

2.  **Loading the Pre-trained Model**: You will load the GPT-2 model and its tokenizer. You\'ll be expected to replace the placeholders in the `# TODO` comments with the appropriate code to load the model and tokenizer.

3.  **Model Evaluation Mode**: Before you start the exploration, you will set the model to evaluation mode to disable dropout and other training-specific behaviors that could affect the outcome.

4.  **Input Preparation**: Define and encode the input text \"I have a dream\" so that it can be processed by the model.

5.  **Computing Original Loss**: Compute and store the original loss of the model without any perturbations to the parameters. This will serve as a reference point for comparison.

6.  **Defining Random Directions**: You will create two sets of random directions that are orthogonal to each other for each parameter of the model. These directions represent random perturbations in the parameter space.

7.  **Normalizing Vectors**: To ensure fair comparison, you will normalize the random directions so that they have the same scale as the model parameters.

8.  **Exploring the Loss Landscape**: You will define a grid in the 2D space formed by the two random directions. For each point in the grid, you will perturb the model parameters in the direction of the grid point and compute the resulting loss.

In other words, you should treat the high-dimensional space of the parameters as a 2D space that is spanned from the two random directions. You will plot this 2D space, using the third dimension to show the output of the loss function.

9.  **Visualizing the Results**: After computing the losses for each point in the grid, you will visualize the loss landscape using Plotly to create a 3D plot. This will give you a visual representation of how the loss changes as you move in different directions in the parameter space.

Your task is to fill in the `# TODO` sections with the correct code to complete the exercise. This will involve encoding inputs, computing loss, perturbing model parameters, and reverting them back to their original state after each computation. Finally, you will create a 3D plot to visualize the loss landscape.

This exercise will help you understand the complexity of the model\'s loss surface and the impact of parameter perturbations on model performance. It is a great way to visualize what is often thought of as an abstract concept in the study of neural networks and deep learning.

Once you have completed the exercise, you will be better equipped to understand the challenges in optimizing large language models and the importance of the directions in which the parameters are updated during training.

Further Reading:
- [Visualizing the Loss Landscape of Neural Nets](https://arxiv.org/abs/1712.09913)


Here is yet another explanation of the 8th bullet point above. Do not read it if you already understand what to do.

-   You have a model with many parameters. You want to understand how small changes to those parameters affect the model\'s performance, measured by a \"loss\" function (a lower loss means better performance).
-   To make this manageable, you\'ll pick just two random directions in the vast space where all the parameters exist. Think of these directions as two arrows pointing different ways from a starting point.
-   Next, you\'ll create a grid that covers a range of positions along these two directions. Each point on this grid represents a unique combination of adjustments along the two arrows.
-   For every point on the grid, you will slightly change the model\'s parameters in the specific way that point suggests. Then you\'ll measure the model\'s performance (i.e., calculate the loss) with these adjusted parameters.
-   Finally, you will create a 3D plot: the first two dimensions are the grid points (the adjustments in the two random directions), and the third dimension is the loss at each point. This plot will show you a \'landscape\' of the model\'s performance, indicating how sensitive the model is to changes in these two random directions.

In [ ]:
!pip install -q transformers

In [ ]:
def gram_schmidt_orthogonalization(parameters):
    orthogonal_parameters = []

    for param in parameters:
        flattened_param = param.reshape(-1)
        random_vector = torch.randn_like(flattened_param)
        orthogonal_param = random_vector - (flattened_param.dot(random_vector) / flattened_param.dot(flattened_param)) * flattened_param
        orthogonal_param = orthogonal_param.reshape(param.shape)
        orthogonal_parameters.append(orthogonal_param.detach().clone())

    return orthogonal_parameters

In [ ]:
%%time

import torch
import numpy as np
import plotly.graph_objects as go
from tqdm import tqdm
import imageio
import os
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# Load pre-trained model
model_name = 'gpt2'
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

# Set model to evaluation mode
model.eval()

for param in model.parameters():
    param.requires_grad = False

# Define our input
input_text = "I have a dream"
inputs = tokenizer(input_text, return_tensors="pt")

# Compute the original loss
outputs = model(**inputs, labels=inputs["input_ids"])
original_loss = outputs.loss.item()

# Define two orthogonal random directions for each model parameter
direction1 = [torch.randn_like(p) for p in model.parameters()]
direction2 = gram_schmidt_orthogonalization(direction1)


# Normalize vectors
for p, d1, d2 in zip(model.parameters(), direction1, direction2):
    norm_p = torch.linalg.norm(p.flatten())
    d1.div_(torch.linalg.norm(d1.flatten())).mul_(norm_p)
    d2.div_(torch.linalg.norm(d2.flatten())).mul_(norm_p)


# Define the range to explore
x = np.linspace(-1, 1, 20)
y = np.linspace(-1, 1, 20)
X, Y = np.meshgrid(x, y)

# Prepare to collect the losses
Z = np.zeros((len(x), len(y)))

# Compute loss for each direction
for i in tqdm(range(x.size), desc="x progress"):
    for j in tqdm(range(y.size), desc="y progress"):
        initial_parameters = [p.clone().detach() for p in model.parameters()]
        # Perturb the model parameters
        for p, d1, d2 in zip(model.parameters(), direction1, direction2):
            p.data.add_(X[i, j] * d1 + Y[i, j] * d2)

        # Compute the loss
        outputs = model(**inputs, labels=inputs["input_ids"])
        Z[i, j] = outputs.loss.item()

        # Revert the model parameters
        for p, initial_p in zip(model.parameters(), initial_parameters):
            p.data.copy_(initial_p)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
x progress: 100%|██████████| 20/20 [07:19<00:00, 21.97s/it]

CPU times: user 5min 4s, sys: 2min 20s, total: 7min 25s
Wall time: 7min 39s


In [ ]:
# Check whether direction1 and direction2 are really orthogonal, using cosine similarity
from torch.nn.functional import cosine_similarity
sims = []
for i in range(len(direction1)):
    sim = cosine_similarity(direction1[i].flatten(), direction2[i].flatten(), dim=0).item()
    sims.append(sim)
sims = torch.tensor(sims)
print(torch.linalg.norm(sims, ord=1))

tensor(4.3783e-07)


In [ ]:
# Create 3D plot using the meshgrid
fig = go.Figure(data=[go.Surface(z=Z, x=X, y=Y)])

fig.update_layout(scene=dict(xaxis_title='Direction 1', yaxis_title='Direction 2', zaxis_title='Loss'))

fig.show()